# Exploring CNNs

This notebook will help me experiment with CNNs to classify origami photos.

In [1]:
# Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import tensorboard as tb
import psycopg2
from PIL import Image
import os

The data are stored locally in a PostgreSQL database. So let's connect.

In [ ]:
try:
    print('Attempting to connect to PostgreSQL...')
    conn = psycopg2.connect(host='localhost', database='origami', user='postgres', password='postgres')
    cur = conn.cursor()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [ ]:
if cur is not None:
    cur.close()
if conn is not None:
    conn.close()
    print('Database connection closed.')